#### Overview

| Detail Tag | Information |
|------------|-------------|
|Script Name | cdm_load_generic |
|Developed By | Premnath Ghawalkar |
|Input Datasets |<ul><li>u_mfg_production_cdm_sap_notification.cdm_config.cdm_config</li></ul>|
|Output Datasets |<ul><li>SAP Notification datasets </li></ul>|
|Input Data Source |<ul><li>SAP Notification</li><li>CDM SAP Notification</li></ul> |
|Output Data Source |<ul><li>Code Orange 2.0 Delta-lake</li><li>Common Data Model</li></ul> |

#### History

| Date | Created/Modified by | Description |
|:----:|--------------|--------|
|23-Feb-2022 | Premnath Ghawalkar |Initial version |


#### Other Details
- This notebook follows based metadata driven data transformation approach. Metadata along with destination operations are defined in "cdm_config" table.

### Import libraries and set parameters

In [0]:
from pyspark.sql.functions import *


## input parameter Dimension table name
dbutils.widgets.text("DestTableName", "code_list_dim")
dbutils.widgets.text("DestSchemaName", "u_quality_qrdp")
dbutils.widgets.text("Truncate", "1")
dbutils.widgets.text("IsSAP", "1")

p_table_name = dbutils.widgets.get("DestTableName")
p_schema_name = dbutils.widgets.get("DestSchemaName")
p_truncateload = dbutils.widgets.get("Truncate")
p_is_sap = int(dbutils.widgets.get("IsSAP"))

### Set parameters
c_job_loc = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
c_job_name = c_job_loc.rsplit('/',1)[1]

### Logger

In [0]:

%run "/Shared/gsc_cdm_sap_notifications/src/utils/bio_helper_utils"




In [0]:
### set logger
logger = setlogger(app_name=c_job_name, level=logging.INFO)

###  Configure for SAP enterprise mesh node

In [0]:
### set connection string for Enterprise mesh node
# if p_is_sap == 1:
#   enterprise_mesh = configure_mesh_node(scope_name='copscmfgpus6kv001',
#                                       adls_name_secret = 'ent-mesh-adls-name',
#                                       tenant_id_secret = 'ent-mesh-tenant-id',
#                                       client_id_secret = 'ent-mesh-adls-client-id',
#                                       client_key_secret = 'ent-mesh-adls-client-secret')

In [0]:
spark.conf.set("spark.sql.adaptive.enabled",True)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled",True)
spark.conf.set("spark.sql.inMemoryColumnarStorage.compressed", True)
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

Out[8]: DataFrame[key: string, value: string]

In [0]:
def configure_mesh_node(scope_name: str = None,
                        adls_name_secret: str = None,
                        tenant_id_secret: str = None,
                        client_id_secret: str = None,
                        client_key_secret: str = None):
  
    # get secret from key vault
    ent_adls_name = dbutils.secrets.get(scope=scope_name,key=adls_name_secret)
    tenant_id = dbutils.secrets.get(scope=scope_name,key=tenant_id_secret)
    client_id = dbutils.secrets.get(scope=scope_name,key=client_id_secret)
    client_key = dbutils.secrets.get(scope=scope_name,key=client_key_secret)

    # set spark configuration
    spark.conf.set("fs.azure.account.auth.type.{adls}.dfs.core.windows.net".format(adls=ent_adls_name), "OAuth")
    spark.conf.set("fs.azure.account.oauth.provider.type.{adls}.dfs.core.windows.net".format(adls=ent_adls_name),
             "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
    spark.conf.set("fs.azure.account.oauth2.client.endpoint.{adls}.dfs.core.windows.net".format(adls=ent_adls_name),
             "https://login.microsoftonline.com/{tenant}/oauth2/token".format(tenant=tenant_id))
    spark.conf.set("fs.azure.account.oauth2.client.id.{adls}.dfs.core.windows.net".format(adls=ent_adls_name),client_id)
    spark.conf.set("fs.azure.account.oauth2.client.secret.{adls_name}.dfs.core.windows.net".format(adls_name=ent_adls_name),client_key)
    
    return None

In [0]:
enterprise_mesh = configure_mesh_node(scope_name='copscmfgpus6kv001',
                                      adls_name_secret = 'ent-mesh-adls-name',
                                      tenant_id_secret = 'ent-mesh-tenant-id',
                                      client_id_secret = 'ent-mesh-adls-client-id',
                                      client_key_secret = 'ent-mesh-adls-client-secret')

###  Read and gather parameters from config table

In [0]:
config_table_df = spark.sql("""  
select * from {schema}.cdm_config where 
active = 1 and
dest_schema_name = '{schema}' and
dest_table_name = '{table_name}'""".format(schema=p_schema_name, table_name=p_table_name))

In [0]:
display(config_table_df)

config_sk dest_table_name dest_table_primary_key dest_schema_name temp_table_name delta_sql_query merge_sql_query subject_area active 10 code_list_dim null u_quality_qrdp temp_code_list_dim SELECT * from (
select distinct x.* , from_utc_timestamp(current_timestamp(),'Europe/London') as record_eff_from_dt,
 cast(null as timestamp) as record_eff_to_dt, 
 cast (1 as integer) active_record_ind,
cast('SAP' as string) source_system_cd ,
sha2(concat_ws(',', x.category_nm,
x.code,
x.code_descr,
x.group_cd,
x.group_descr,
x.catalog_cd,
x.col_1,
x.col_2,
x.col_3,
x.col_4 ),256) as checksum ,
 cast('I' as string ) as delta_flag,
from_utc_timestamp(current_timestamp(),'Europe/London') as f_last_update
from (

select * from
(
select
distinct
'Notification Code' as category_nm, 
QN.qmcod code,
cd.KURZText code_descr, 
QN.qmgrp group_cd,
cdg.KURZText group_descr, 
QN.QMKAT catalog_cd,
cast(null as string) as col_1,
cast(null as string) as col_2,
cast(null as string) as col_3,
cast(null as string) as col_4
from delta.`abfss://root@[REDACTED].dfs.core.windows.net/data/trusted/erp-nonconf/manufacturing-quality/global/sap/t_erp_manufacturing_quality_rbp_nonconf/qmel` QN
join delta.`abfss://root@[REDACTED].dfs.core.windows.net/data/trusted/erp-nonconf/manufacturing-quality/global/sap/t_erp_manufacturing_quality_rbp_nonconf/qmel` QML on QN.qmnum=QML.qmnum and QN.mandt =QML.mandt
left join 
delta.`abfss://root@[REDACTED].dfs.core.windows.net/data/trusted/erp-nonconf/manufacturing-quality/global/sap/t_erp_manufacturing_quality_rbp_nonconf/qpct` cd 
on cd.codegruppe=QN.QMGRP AND cd.code=QN.QMCOD AND cd.sprache='E' and QN.QMKAT=cd.KATALOGART and cd.mandt=qn.mandt
left join 
delta.`abfss://root@[REDACTED].dfs.core.windows.net/data/trusted/erp-nonconf/manufacturing-quality/global/sap/t_erp_manufacturing_quality_rbp_nonconf/qpgt` cdg on cdg.codegruppe=QN.QMGRP AND cdg.sprache='E' and QN.QMKAT=cdg.KATALOGART and cdg.mandant=qn.mandt

union all

select 
distinct
'Defect Location' as category_nm,
QNI.oteil code,
cdl.KURZText code_descr,
QNI.otgrp group_cd, 
cdlg.KURZText group_descr,
QNI.OTKAT as catalog_cd,
cast(null as string) as col_1,
cast(null as string) as col_2,
cast(null as string) as col_3,
cast(null as string) as col_4
from delta.`abfss://root@[REDACTED].dfs.core.windows.net//data/trusted/erp-nonconf/manufacturing-quality/global/sap/t_erp_manufacturing_quality_rbp_nonconf/qmfe` QNI 
left join 
delta.`abfss://root@[REDACTED].dfs.core.windows.net/data/trusted/erp-nonconf/manufacturing-quality/global/sap/t_erp_manufacturing_quality_rbp_nonconf/qpct` cdl on cdl.codegruppe=QNI.OTGRP AND cdl.code=QNI.oteil AND cdl.sprache='E' and QNI.OTKAT=cdl.KATALOGART and cdl.mandt=QNI.mandt
left join delta.`abfss://root@[REDACTED].dfs.core.windows.net/data/trusted/erp-nonconf/manufacturing-quality/global/sap/t_erp_manufacturing_quality_rbp_nonconf/qpgt` cdlg on cdlg.codegruppe=QNI.OTGRP AND cdlg.sprache='E' and QNI.OTKAT=cdlg.KATALOGART and cdlg.mandant=QNI.mandt

union all

select 
distinct
'Defect Type' as category_nm,
QNI.fecod code,
cd.KURZText code_descr, 
QNI.fegrp group_cd,
cdg.KURZText group_descr,
QNI.FEKAT as catalog_cd,
cast(null as string) as col_1,
cast(null as string) as col_2,
cast(null as string) as col_3,
cast(null as string) as col_4
from delta.`abfss://root@[REDACTED].dfs.core.windows.net//data/trusted/erp-nonconf/manufacturing-quality/global/sap/t_erp_manufacturing_quality_rbp_nonconf/qmfe` QNI 
join 
delta.`abfss://root@[REDACTED].dfs.core.windows.net/data/trusted/erp-nonconf/manufacturing-quality/global/sap/t_erp_manufacturing_quality_rbp_nonconf/qmel` QML on QNI.qmnum=QML.qmnum and QNI.mandt =QML.mandt
left join 
delta.`abfss://root@[REDACTED].dfs.core.windows.net/data/trusted/erp-nonconf/manufacturing-quality/global/sap/t_erp_manufacturing_quality_rbp_nonconf/qpct` cd on cd.codegruppe=QNI.FEGRP AND cd.code=QNI.FECOD AND cd.sprache='E' and QNI.FEKAT=cd.KATALOGART and cd.mandt=QNI.mandt
left join 
delta.`abfss://root@[REDACTED].dfs.core.windows

###  Execute source delta query and merge query

In [0]:
temp_table_name = config_table_df.rdd.collect()
for row in temp_table_name:
    try:
        #create temp view based on temp table name from config table
        src_query = spark.sql("""{query}""".format(query=row["delta_sql_query"]))
        src_query.createOrReplaceTempView(row["temp_table_name"])
        if p_truncateload=='1':
            table_action_query = spark.sql("""{query}""".format(query="TRUNCATE table "+ row["dest_schema_name"]+"."+row["dest_table_name"]   ))
        else:
            print('No Truncate data')
        # execute merge query
        merge_query = spark.sql("""{merge_query}""".format(merge_query=row["merge_sql_query"]))
    except Exception as e:
        msg = 'ERROR: While processing dimension "{dim}", encountered - "{error}" as -'.format(dim=str(row["dest_schema_name"]+'.'+row["dest_table_name"]), error=str(e))
        log(msg, logger)